In [1]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()

In [2]:
data_dir = 'data' 
prefix = 'rcmdKNN'
input_data = sagemaker_session.upload_data(data_dir, key_prefix=prefix)

In [3]:
from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point='train.py',
                    framework_version='0.23-1',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    py_version='py3',
                    source_dir='source',
                    image_uri=None,
                    hyperparameters = {'n_neighbors':10,
                                       'metric':'cosine',
                                       'algorithm':'brute'})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [4]:
estimator.fit({'train': input_data})

2021-12-13 16:24:34 Starting - Starting the training job...
2021-12-13 16:24:38 Starting - Launching requested ML instancesProfilerReport-1639412674: InProgress
......
2021-12-13 16:25:57 Starting - Preparing the instances for training.........
2021-12-13 16:27:30 Downloading - Downloading input data...
2021-12-13 16:27:50 Training - Downloading the training image..2021-12-13 16:28:16,644 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-12-13 16:28:16,646 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-13 16:28:16,657 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-12-13 16:28:17,081 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-13 16:28:20,109 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-12-13 16:28:20,123 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus inst

In [5]:
from sagemaker.sklearn import SKLearnModel

sklearn_model = SKLearnModel(model_data=estimator.model_data,
                             role=role,
                             entry_point="predict.py",
                             source_dir='source',
                             framework_version='0.23-1')

predictor = sklearn_model.deploy(instance_type="ml.m4.xlarge", initial_instance_count=1)

----------!

In [6]:
import scipy
import numpy as np
train_data = scipy.sparse.load_npz('./data/artist_matrx.npz')
train_data = np.array(train_data.todense())

predictor.predict(train_data[0].reshape(1,-1))

array([  0, 311,   5,  70,  63,   4,  18, 269, 774, 113])